## 1: Desenvolva uma descrição completando a tabela abaixo sobre o ambiente de tarefas de cada um dos agentes a seguir. 

> (A) Agente comentarista de jogo
>
> (B) Agente sugestor de vinho
>
> (C) Agente treinador de time, ex. de basquete

| Agentes | Medida de desempenho | Ambiente | Atuadores | Sensores |
|---------|----------------------|----------|-----------|----------|
| A | Popularidade de seus comentários | Campeonatos, partidas ou noticiários sobre jogos | Comentar, analisar e comparar jogadas, partidas e jogadores | Dados, gravações, áudios do jogo |
| B | Classificação de desempenho pelos clientes | Lojas ou fábricas de vinho | Encaminhar clientes para diferentes tipos de vinho | Histórias, gostos, pedidos do cliente |
| C | Proporção de vitórias/derrotas do time | Clubes de esporte | Condicionar os jogadores, dar diferentes objetivos e rotinas para cumprirem | Estatísticas, resultados de treinos |

## 2: Diferencie os 4 tipos de aprendizado de máquina. Para tanto, escolha uma base de dados do Kaggle e explique como aplicar os 4 métodos nessa mesma base.

## 3: Utilize o método k-NN na base escolhida na questão anterior para um contexto SUPERVISIONADO e para classificação.

### a) Aplique processos necessários para o pré-processamento e execução do método k-NN.

### b) Aplique correlação ou PCA, selecione um número menor de atributos com esses métodos e justifique sua decisão.

### c) Escolha 5 valores de K e mostre os resultados (escolha 3 métricas de avaliação e mostre a matriz de confusão). Explique os resultados encontrados.

### d) Escolha 2 métricas de similaridade (diferentes distâncias ou outras métricas - correlação, por exemplo) para definir o(s) vizinho(s) mais próximos e explique os resultados encontrados.

### e) Aplique k-NN no conjunto de atributos do item 2 usando o melhor k e a melhor métrica de distância.

## 4: Considerando a matriz de confusão abaixo, quais são os valores das métricas listadas abaixo?

Colunas: verdadeiras

Linhas: preditas

| Classes | Maçã | Laranja | Limão | Manga |
|---------|------|---------|-------|-------|
| Maçã    | 70   | 20      | 5     | 5     |
| Laranja | 10   | 67      | 3     | 20    |
| Limão   | 5    | 30      | 55    | 10    |
| Manga   | 5    | 4       | 2     | 1000  |

- Sensibilidade
- Especificidade
- Precisão
- Medida F1
- Acurácia
- Coeficiente de Matthews
- AUC

### Como você resolveria o problema dessa base de dados observando as métricas de avaliação?

## 5: Elabore uma pergunta e responda sobre "Preconceito e Inteligência Artificial", refletindo aspectos importantes do método que uma pessoa engenheiro de dados/cientista de dados deve conhecer e se preocupar garantindo ética nos modelos criados.